In [133]:
###### 
# 
# Script to create a messy concatenated.csv document with all the information from the guide.pdf documents.
# 
###### 


import fitz
import re
from pathlib import Path
import pandas as pd


# convert PDF guides to txt

path = "./files/UKDS_Versions-consolidated/guides-pdf"
output = "./files/UKDS_Versions-converted/guides"
OVERWRITE = True

for pdf in Path(path).glob("*.pdf"):
    doc = fitz.open(pdf)  # open document
    text = ""
    for page in doc:  # iterate the document pages
        text += page.get_text() # .encode("utf8")  # get plain text (is in UTF-8)

    # strip the beginning
    text = text[text.find("of the page images. ")+20:]

    output = Path(output)
    output.mkdir(parents=True, exist_ok=True)
    txt_doc = output / pdf.name.replace(".pdf", ".txt")
    if not txt_doc.exists() or OVERWRITE:
        txt_doc.write_text(text)


    # parse
    lines = [(x.startswith("SN "), x) for x in text.splitlines()]

    category = None
    _lines = []
    for line in lines:
        is_sn, text = line
        
        if is_sn:
            sn = text
            continue

        if text == "------ ":
            continue

        if text.strip() == "":
            continue

        if re.match(r"^Page \d+ of ", line[1]):
            continue

        _lines.append(text)

    lines = _lines



    item_id = 0
    items = {}
    new = False
    hold = ""
    for i, e in reversed(list(enumerate([("...." in x, x) for x in lines]))):
        has_cat, text = e
        if "----" in text:
            if item_id in items and not items[item_id]:
                pass
            else:
                item_id += 1
            items[item_id] = {}
            new = True
            continue
        if not has_cat:
            if new:
                items[item_id]["Name"] = text
                new = False
                continue
            new = False
            hold = text + hold
            continue
        else:
            new = False
            category, _text = re.split(r"\.{4,200}", text)
            category = category.strip()
            items[item_id][category] = _text + hold
            hold = ""

    csv_doc = output / pdf.name.replace(".pdf", ".csv")
    df = pd.DataFrame(items).T
    df["id"] = pdf.stem
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    df.to_csv(csv_doc)

df = pd.concat([pd.read_csv(f) for f in Path(output).glob("**/*.csv") ], ignore_index=True)
df.to_csv(output / "concatenated.csv")

In [148]:
doc = fitz.open("./files/UKDS_Versions-consolidated/pdf/BCL12001.pdf")  # open document
for idx, page in enumerate(doc):  # iterate the document pages
    blocks = page.get_text_blocks()
    if len(blocks) > 1:
        print(idx)
    if idx > 0:
        continue

In [146]:
doc[2].get_text_blocks()

[(0.0,
  0.0,
  538.5599975585938,
  779.5800170898438,
  '<image: DeviceGray, width: 2992, height: 4331, bpc: 1>',
  0,
  1)]

In [ ]:
page.get_text_blocks

In [1]:
import detectron2
import layoutparser as lp
from cv2 import imread

ModuleNotFoundError: No module named 'detectron2'

In [2]:
image = imread("./files/UKDS_Versions/UKDA-5101-txt/txt/bcl15027/tiff/0005_bcl15027_tif_00009zo4.tif")

In [5]:
lp.is_detectron2_available()

False

In [3]:
model = lp.Detectron2LayoutModel(
            config_path ='lp://NewspaperNavigator/faster_rcnn_R_50_FPN_3x/config', # In model catalog
            label_map   ={0: "Photograph", 1: "Illustration", 2: "Map", 3: "Comics/Cartoon", 4: "Editorial Cartoon", 5: "Headline", 6: "Advertisement"}, # In model`label_map`
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8] # Optional
        )
model.detect(image)

AttributeError: module layoutparser has no attribute Detectron2LayoutModel

In [44]:
sn

'SN 5112 - Digital Library of Historical Directories, 1750-1919: Derbyshire,  '

In [49]:
for line in lines:
    if line[0] == None:
        print(line)

(None, 'Leicestershire, Nottinghamshire and Rutland, 1881-1904 ')


In [26]:
cat_line

[]

In [9]:
import re

DIVIDER = re.compile("^-{2,200}")

In [73]:
# test for divider
print(DIVIDER.match("dsfdsf-------"))
print(DIVIDER.match("--dsfdsf-------"))


None
<re.Match object; span=(0, 2), match='--'>


In [76]:
CAT = re.compile("^\w\.{3,200}")

In [83]:
for txt in output.glob("*.txt"):
    text = txt.read_text()
    lines = text.splitlines()

In [85]:
lines

['Page 1 of 16 ',
 "Source ............. Liverpool's First Directory: a reprint of the ",
 "names and addresses from Gore's Directory for 1766; to which is added a ",
 'street directory for the same year, compiled by George T. Shaw & ',
 'Isabella Shaw. Also, a history of the Liverpool directories from 1766 ',
 'to 1907 by George T. Shaw ',
 'Published .......... Liverpool: Young, 1907 ',
 'Repository ......... University of Leicester Library, P.O. Box 248, ',
 'University Road, Leicester, LE1 9QD ',
 'Volume Note ........ 1907 reprint of the original 1766 volume. ',
 'Digitisation Note .. ',
 'Headings ........... Title Page, Preface & Contents // History of ',
 "Liverpool Directories // Gore's Directory, 1766 // Streets, 1766 // ",
 'Professions, 1766 ',
 'Content Note ....... ',
 'Features ........... Illustration of John Gore ',
 'Features Note ...... ',
 'ID ................. LUL6015 ',
 'Image Files ........ 81 files totalling 5.38 MB in folder LUL6015tif ',
 'Text File .........

[]

In [13]:
import cv2
import pytesseract
from pathlib import Path

img = cv2.imread("./files/UKDS_Versions/UKDA-5101-txt/txt/bcl8001/tiff/0002_bcl8001_tif_000039v7.tif")
#img = cv2.resize(img, None, fx=3, fy=3)

#img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [14]:
Path("test.txt").write_text(pytesseract.image_to_string(img))

1419